In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-02-25 11:10:04--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  68.5MB/s    in 0.4s    

2025-02-25 11:10:05 (68.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [14]:
# Clean data - remove users whith less than 200 ratings
print(df_ratings)
df_filtered_ratings = df_ratings[df_ratings.groupby('user')['rating'].transform('count') >= 200]
print(df_filtered_ratings)

<class 'pandas.core.frame.DataFrame'>
Index: 527556 entries, 1456 to 1147616
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   user    527556 non-null  int32  
 1   isbn    527556 non-null  object 
 2   rating  527556 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 12.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 527556 entries, 1456 to 1147616
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   user    527556 non-null  int32  
 1   isbn    527556 non-null  object 
 2   rating  527556 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 12.1+ MB


In [5]:
# Create new dataframe
rating_count = df_ratings.groupby('isbn')['rating'].count()
rating_avg = df_ratings.groupby('isbn')['rating'].mean()
new_ratings_df = pd.DataFrame({'ratings_count':rating_count, 'rating_avg': rating_avg})
df = df_books.merge(new_ratings_df, on='isbn')

,isbn,title,author,ratings_count,rating_avg
0,0195153448,Classical Mythology,Mark P. O. Morford,1,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,14,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,3,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,11,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1,0.000000


In [ ]:
# Prepare features
x = df[['count_of_ratings', 'avg_rating']]

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()